<a href="https://colab.research.google.com/github/Serurays/Sample_Detection_Tracking_Yolov8_DeepSORT_Robotics/blob/main/Color_Sample_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Required Libraries**

In [ ]:
!pip install ultralytics xmltodict

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.5/913.5 kB 28.2 MB/s eta 0:00:00


___

# **(Optional) Mount Google Drive**

If you want to save the results in your drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


___

# **Convert CVAT annotations to YOLO annotations**

In [ ]:
import os
import random
import shutil
import xml.etree.ElementTree as ET
from ultralytics import YOLO

IMAGES_DIR = "images"
XML_FILE   = "annotations.xml"
OUTPUT_DIR = "/content/drive/MyDrive/Robotics_Color_Sample_Detection/data_yolo"
TRAIN_RATIO = 0.8

# Class mapping for your dataset (label_name -> class_index).
# The order here is critical. We use:
# 0: red, 1: blue, 2: yellow
CLASS_MAP = {
    "red": 0,
    "blue": 1,
    "yellow": 2
}

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
def convert_to_yolo_bbox(xtl, ytl, xbr, ybr, img_width, img_height):
    """
    Convert CVAT bounding box [xtl, ytl, xbr, ybr]
    to YOLO bounding box [x_center, y_center, width, height] in relative coords.
    """
    x_center = ((xtl + xbr) / 2.0) / img_width
    y_center = ((ytl + ybr) / 2.0) / img_height
    w = (xbr - xtl) / img_width
    h = (ybr - ytl) / img_height
    return x_center, y_center, w, h

In [ ]:
# Create train/val folders for images and labels
train_img_dir = os.path.join(OUTPUT_DIR, "images", "train")
val_img_dir   = os.path.join(OUTPUT_DIR, "images", "val")
train_lbl_dir = os.path.join(OUTPUT_DIR, "labels", "train")
val_lbl_dir   = os.path.join(OUTPUT_DIR, "labels", "val")

os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(train_lbl_dir, exist_ok=True)
os.makedirs(val_lbl_dir, exist_ok=True)

In [ ]:
tree = ET.parse(XML_FILE)
root = tree.getroot()

image_elements = root.findall('image')

print("Number of images in XML:", len(image_elements))

Number of images in XML: 87


In [ ]:
# Shuffle
image_elements = list(image_elements)
random.shuffle(image_elements)

train_count = int(len(image_elements) * TRAIN_RATIO)
train_elements = image_elements[:train_count]
val_elements   = image_elements[train_count:]
print(f"Train images: {len(train_elements)}, Val images: {len(val_elements)}")

Train images: 69, Val images: 18


In [ ]:
def process_images(image_subset, subset_name):
    """
    For each image in the subset, copy the image file to the appropriate train/val folder,
    create a YOLO label file with bounding boxes, and place it in labels/train or labels/val.
    """
    if subset_name == "train":
        img_out_dir = train_img_dir
        lbl_out_dir = train_lbl_dir
    else:
        img_out_dir = val_img_dir
        lbl_out_dir = val_lbl_dir

    for img_elem in image_subset:
        # Extract metadata
        file_name = img_elem.attrib['name']
        width = float(img_elem.attrib['width'])
        height = float(img_elem.attrib['height'])

        # Source path for the image
        src_img_path = os.path.join(IMAGES_DIR, file_name)
        # Destination path
        dst_img_path = os.path.join(img_out_dir, file_name)

        if not os.path.exists(src_img_path):
            print(f"Warning: {src_img_path} not found. Skipping.")
            continue

        # Copy image to the appropriate folder
        shutil.copy2(src_img_path, dst_img_path)

        # Prepare label lines
        boxes = img_elem.findall('box')
        label_lines = []

        for b in boxes:
            label_str = b.attrib['label']
            xtl = float(b.attrib['xtl'])
            ytl = float(b.attrib['ytl'])
            xbr = float(b.attrib['xbr'])
            ybr = float(b.attrib['ybr'])

            # Convert label to class index
            if label_str not in CLASS_MAP:
                print(f"Warning: Label '{label_str}' not in CLASS_MAP. Skipping.")
                continue
            class_idx = CLASS_MAP[label_str]

            x_center, y_center, w, h = convert_to_yolo_bbox(xtl, ytl, xbr, ybr, width, height)
            label_line = f"{class_idx} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}"
            label_lines.append(label_line)

        # Write the labels to a .txt file
        txt_file_name = os.path.splitext(file_name)[0] + ".txt"
        txt_out_path = os.path.join(lbl_out_dir, txt_file_name)
        with open(txt_out_path, "w") as f:
            for line in label_lines:
                f.write(line + "\n")

# Process train and val
process_images(train_elements, "train")
process_images(val_elements, "val")

print("Finished processing images and labels.")

Finished processing images and labels.


___

# **Create data.yaml**

In [ ]:
# Sort the CLASS_MAP by index for consistent ordering
sorted_classes = sorted(CLASS_MAP.items(), key=lambda x: x[1])
# e.g. [('red', 0), ('blue', 1), ('yellow', 2)]

data_yaml_path = os.path.join(OUTPUT_DIR, "dataset.yaml")
with open(data_yaml_path, "w") as f:
    f.write(f"train: {os.path.abspath(train_img_dir)}\n")
    f.write(f"val: {os.path.abspath(val_img_dir)}\n")
    f.write("names:\n")
    for k, v in sorted_classes:
        f.write(f"  {v}: {k}\n")

print(f"dataset.yaml created at {data_yaml_path}")

dataset.yaml created at /content/drive/MyDrive/Robotics_Color_Sample_Detection/data_yolo/dataset.yaml


___

# **Training**

In [ ]:
# Load the YOLO model.
model = YOLO('yolov8s.pt')

# Train
results = model.train(
    data=data_yaml_path,
    epochs=1000,
    imgsz=640,
    batch=32,
    name="robotics_model"
)

print("Training completed. Check 'runs/detect/my_custom_model' for logs and weights.")

Ultralytics 8.3.67 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Robotics_Color_Sample_Detection/data_yolo/dataset.yaml, epochs=1000, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=robotics_model2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False

train: Scanning /content/drive/MyDrive/Robotics_Color_Sample_Detection/data_yolo/labels/train.cache... 69 images, 4 backgrounds, 0 corrupt: 100%|██████████| 69/69 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/Robotics_Color_Sample_Detection/data_yolo/labels/val.cache... 18 images, 2 backgrounds, 0 corrupt: 100%|██████████| 18/18 [00:00<?, ?it/s]


Plotting labels to runs/detect/robotics_model2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/robotics_model2
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

   180/1000      7.99G     0.5154      0.384     0.9118         11        640: 100%|██████████| 3/3 [00:01<00:00,  2.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.61it/s]

                   all         18         32      0.978      0.996      0.995      0.871


EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 80, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

180 epochs completed in 0.166 hours.
Optimizer stripped from runs/detect/robotics_model2/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/robotics_model2/weights/best.pt, 22.5MB

Validating runs/detect/robotics_model2/weights/best.pt...
Ultralytics 8.3.67 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.48it/s]


                   all         18         32      0.984          1      0.995      0.891
                   red         10         13          1          1      0.995       0.86
                  blue          4          7      0.969          1      0.995      0.903
                yellow          9         12      0.984          1      0.995      0.911
Speed: 0.2ms preprocess, 4.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/robotics_model2
Training completed. Check 'runs/detect/my_custom_model' for logs and weights.


___

# **Inference**

In [ ]:
# Path to the best weights saved by YOLO during training
trained_weights = "best.pt"
model = YOLO(trained_weights)

# Predict on all images in the validation folder
results = model.predict(
    source=val_img_dir,
    conf=0.7,
    save=True
)

print(f"Inference completed on all images in: {val_img_dir}")
print("Results saved in 'runs/predict' folder.")

___

# **(Optional) Save YOLO Results**

In [ ]:
import shutil

shutil.make_archive("runs", "zip", "runs")

'/content/runs.zip'

___

# **Tracking**

After updating the DeepSort Code using my Medium article which you can find in ReadMe file, upload it as zipped to Google Colab to do tracking, if you don't have a local GPU, if you have one, then you can directly run these code chunks locally.

In [ ]:
!unzip deep_sort.zip

Archive:  deep_sort.zip
   creating: deep_sort/
  inflating: deep_sort/.gitignore    
   creating: deep_sort/application_util/
  inflating: deep_sort/application_util/image_viewer.py  
  inflating: deep_sort/application_util/preprocessing.py  
  inflating: deep_sort/application_util/visualization.py  
 extracting: deep_sort/application_util/__init__.py  
   creating: deep_sort/deep_sort/
  inflating: deep_sort/deep_sort/detection.py  
  inflating: deep_sort/deep_sort/iou_matching.py  
  inflating: deep_sort/deep_sort/kalman_filter.py  
  inflating: deep_sort/deep_sort/linear_assignment.py  
  inflating: deep_sort/deep_sort/nn_matching.py  
  inflating: deep_sort/deep_sort/track.py  
  inflating: deep_sort/deep_sort/tracker.py  
 extracting: deep_sort/deep_sort/__init__.py  
   creating: deep_sort/deep_sort/__pycache__/
  inflating: deep_sort/deep_sort/__pycache__/detection.cpython-312.pyc  
  inflating: deep_sort/deep_sort/__pycache__/iou_matching.cpython-312.pyc  
  inflating: deep_so

In [ ]:
import cv2
import os
import random
import numpy as np

from ultralytics import YOLO

# Deep SORT imports
from deep_sort.deep_sort.tracker import Tracker as DeepSortTracker
from deep_sort.tools import generate_detections as gdet
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection

print("Imports done.")

Imports done.


In [ ]:
class Track:
    """
    A simple container to hold track_id and bbox data.
    """
    def __init__(self, track_id, bbox):
        self.track_id = track_id
        self.bbox = bbox


class DeepSortWrapper:
    """
    Wrapper class around the Deep SORT functionality.
    """
    def __init__(self, model_filename='model_data/mars-small128.pb', max_cosine_distance=0.4, nn_budget=None):
        metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
        self.tracker = DeepSortTracker(metric)
        self.encoder = gdet.create_box_encoder(model_filename, batch_size=1)
        self.tracks = []

    def update(self, frame, detections):
        """
        Args:
            frame (numpy.ndarray): Current video frame (BGR).
            detections (list): List of detections in [x1, y1, x2, y2, score] format.
        """

        # Step 1: If no detections, just run tracker predict
        if len(detections) == 0:
            self.tracker.predict()
            self.tracker.update([])
            self._update_tracks()
            return

        # Step 2: Convert [x1, y1, x2, y2] to [x, y, w, h]
        bboxes = np.array([d[:4] for d in detections])
        scores = [d[4] for d in detections]

        bboxes[:, 2:] = bboxes[:, 2:] - bboxes[:, :2]  # width, height

        # Step 3: Generate features for each bbox
        features = self.encoder(frame, bboxes)

        # Step 4: Wrap Deep SORT detections
        dets = []
        for bbox_id, bbox in enumerate(bboxes):
            dets.append(Detection(bbox, scores[bbox_id], features[bbox_id]))

        # Step 5: Run the tracker
        self.tracker.predict()
        self.tracker.update(dets)
        self._update_tracks()

    def _update_tracks(self):
        """
        Refresh self.tracks with updated tracks from the deep_sort tracker
        """
        active_tracks = []
        for track in self.tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            bbox = track.to_tlbr()  # returns [x1, y1, x2, y2]
            track_id = track.track_id
            active_tracks.append(Track(track_id, bbox))

        self.tracks = active_tracks

In [ ]:
yolo_model_path = "best.pt"
model = YOLO(yolo_model_path)

# Initialize the Deep SORT wrapper
deepsort = DeepSortWrapper(
    model_filename='mars-small128.pb',
    max_cosine_distance=0.4,
    nn_budget=None
)

print("YOLO model and Deep SORT wrapper initialized.")

YOLO model and Deep SORT wrapper initialized.


In [ ]:
class_names = ["red", "blue", "yellow"]

In [ ]:
import cv2

# Open input video
cap = cv2.VideoCapture("vid.mp4")
if not cap.isOpened():
    print("Could not open video.")
    raise SystemExit

detection_threshold = 0.8

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize VideoWriter to save the output video
output_file = "output_with_detections.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec for .mp4
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

# Process video frames
while True:
    ret, frame = cap.read()
    if not ret:
        print("Finished processing video.")
        break

    # Run YOLO model on the frame
    results = model(frame)

    # Process detections
    all_detections = []
    for r in results:
        for box in r.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = box
            if score >= detection_threshold:
                class_name = class_names[int(class_id)]
                class_counts[class_name] = class_counts.get(class_name, 0) + 1
                all_detections.append([int(x1), int(y1), int(x2), int(y2), float(score)])

    # Update Deep SORT with detections
    deepsort.update(frame, all_detections)

    # Draw bounding boxes and text
    for track in deepsort.tracks:
        x1, y1, x2, y2 = track.bbox
        track_id = track.track_id
        colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(100)]
        color = colors[track_id % len(colors)]
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Overlay YOLO detections summary
    y_offset = 20
    for class_name, count in class_counts.items():
        cv2.putText(frame, f"{count} {class_name}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        y_offset += 30

    # Write the processed frame into the output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video saved as {output_file}")


0: 640x384 1 red, 1 blue, 12.4ms
Speed: 2.2ms preprocess, 12.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 red, 1 blue, 10.7ms
Speed: 4.1ms preprocess, 10.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 red, 1 blue, 11.0ms
Speed: 4.1ms preprocess, 11.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 red, 1 blue, 10.7ms
Speed: 3.3ms preprocess, 10.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 red, 1 blue, 10.7ms
Speed: 3.4ms preprocess, 10.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 red, 1 blue, 11.8ms
Speed: 3.4ms preprocess, 11.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 red, 2 blues, 10.7ms
Speed: 3.4ms preprocess, 10.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 red, 2 blues, 10.7ms
Speed: 3.4ms preprocess, 10.7ms inference, 1.3m